In [1]:
pip install langchain_community 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 24.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langgraph

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 11.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file
openai_api_key = os.getenv("OPENAI_API_KEY")


from langchain.tools import tool
import uuid



from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

- Prompts Definition: Define two different prompt templates for replacing an existing address and adding a new address.
- LLM Instance: Create an instance of the language model (ChatOpenAI).
- Memory Buffer: Use ConversationBufferMemory to handle the chat history.
- LLM Chains: Create LLMChain objects for both prompts, integrating the memory buffer.
- Runnable Objects: Create RunnableLambda objects to wrap the chain runs.
- Parallel Execution: Use RunnableParallel to run both chains in parallel.
- Function Binding: Bind functions for address extraction.
- Agent Executor: Integrate everything into AgentExecutor, using the memory buffer to maintain context.

In [17]:
#depreciated
# Define the combined chat prompt template for replacing an existing address
replace_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing address with an exisiting one. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing address with an exisiting one.\n"
        "5. If the user asks about the conversation so far answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "\n"
        "If the user wants to replace an existing address, follow these steps:\n"
        "Step 1: If the user wants to replace an existing address, ask for the contract ID.\n"
        "Step 2: Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "Step 3: Ask the user to select one address to replace.\n"
        "Step 4: Confirm the choice and generate a reference ID.\n"
        "Step 5: Respond with: 'Great, we have received your request to replace with an existing address. Kindly note your reference ID [reference_id] for further communications.'\n"
        "\n"
        "Example Conversation:\n"
        "User: "
        "Assistant: Would you like to replace address with an existing address?\n"
        "User: Replace address\n"
        "Assistant: Provide me your contract ID.\n"
        "User: ABC1003\n"
        "Assistant: Fetches the list of addresses with the contract ID from a database and displays:\n"
        "1. 123 Main St, Springfield, IL, 62704\n"
        "2. 456 Oak St, Springfield, IL, 62705\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace with an existing address. Kindly note your reference ID abc12234 for further communications.\n"
    ),
    ("user", "{user_input}"),
])

In [4]:
# Define the combined chat prompt template for replacing an existing address using ReACT framework
replace_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing an address from an existing list of addresses. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing an address from an existing list of addresses.\n"
        "5. If the user asks about the conversation so far, answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "7. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
        "8. If the user wants to go back one step, provide the previous stage's information.\n"
        "9. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
        "\n"
        "Example Conversations:\n"
        "\n"
        "Example 1:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: DEF4567\n"
        "Assistant: Fetching the list of addresses associated with contract ID DEF4567...\n"
        "1. 789 Pine St, Springfield, IL, 62706\n"
        "2. 101 Maple St, Springfield, IL, 62707\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID def56789 for further communications.\n"
        "\n"
        "Example 2:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: GHI8901\n"
        "Assistant: Fetching the list of addresses associated with contract ID GHI8901...\n"
        "1. 234 Elm St, Springfield, IL, 62708\n"
        "2. 345 Cedar St, Springfield, IL, 62709\n"
        "User: What time does the office close?\n"
        "Assistant: Please stay within the context of the conversation. Would you like to replace an address? Please select one address from the list.\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID ghi89101 for further communications.\n"
        "\n"
        "Example 3:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: JKL1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID JKL1234...\n"
        "1. 456 Birch St, Springfield, IL, 62710\n"
        "2. 567 Walnut St, Springfield, IL, 62711\n"
        "User: What did we talk about earlier?\n"
        "Assistant: We were discussing replacing your address. You provided contract ID JKL1234. Please select one address from the list.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID jkl12345 for further communications.\n"
        "\n"
        "Example 4:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: MNO5678\n"
        "Assistant: The provided contract ID does not match any records. Please check the contract ID and provide it again.\n"
        "User: MNO8765\n"
        "Assistant: Fetching the list of addresses associated with contract ID MNO8765...\n"
        "1. 678 Oak St, Springfield, IL, 62712\n"
        "2. 789 Ash St, Springfield, IL, 62713\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID mno87654 for further communications.\n"
        "\n"
        "Example 5:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: PQR9012\n"
        "Assistant: Fetching the list of addresses associated with contract ID PQR9012...\n"
        "1. 890 Fir St, Springfield, IL, 62714\n"
        "2. 901 Spruce St, Springfield, IL, 62715\n"
        "User: Can you confirm the details of address 1?\n"
        "Assistant: The address is 890 Fir St, Springfield, IL, 62714. Please confirm if you would like to replace with this address.\n"
        "User: Yes, replace with address 1.\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID pqr90123 for further communications.\n"
        "\n"
        "Example 6:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: STU3456 and the new address is 345 Pine St\n"
        "Assistant: Please provide only your contract ID to proceed with replacing an existing address.\n"
        "User: STU3456\n"
        "Assistant: Fetching the list of addresses associated with contract ID STU3456...\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID stu34567 for further communications.\n"
        "\n"
        "Example 7:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: VWX6789\n"
        "Assistant: Fetching the list of addresses associated with contract ID VWX6789...\n"
        "1. 234 Birch St, Springfield, IL, 62718\n"
        "2. 345 Cedar St, Springfield, IL, 62719\n"
        "User: Nevermind, I don't want to change it.\n"
        "Assistant: No problem. If you need further assistance, please let me know.\n"
        "\n"
        "Example 8:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: YZA1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID YZA1234...\n"
        "1. 567 Elm St, Springfield, IL, 62720\n"
        "2. 678 Pine St, Springfield, IL, 62721\n"
        "User: Are you sure these are the correct addresses?\n"
        "Assistant: Yes, these are the addresses associated with contract ID YZA1234. Please select one address to replace.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID yza12345 for further communications.\n"
        "\n"
        "Example 9:\n"
        "User: I don't understand.\n"
        "Assistant: I'm here to assist you. I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 10:\n"
        "User: Please repeat.\n"
        "Assistant: Certainly! I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 11:\n"
        "User: Can you tell me what was done so far?\n"
        "Assistant: Sure, we have discussed the process of replacing an address associated with a contract ID. The provided contract ID was contract_id. We fetched the list of addresses associated with this contract ID, and you selected an address to replace. If you have further questions, please let me know.\n"
        "\n"
        "Example 12:\n"
        "User: Can we go back one step?\n"
        "Assistant: Sure, let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the address you would like to replace by entering the corresponding number.\n"
        "\n"
        "Example 13:\n"
        "User: I forgot my contract ID.\n"
        "Assistant: No problem. Please check your records and provide your contract ID so we can proceed with replacing the address.\n"
        "If you cannot find your contract ID, please contact support for further assistance.\n"
        "\n"
        "Example 14:\n"
        "User: How do you know it's not correct?\n"
        "Assistant: The provided contract ID did not match any records in our system. Please double-check your contract ID and provide it again.\n"
        "\n"
        "Example 15:\n"
        "User: Yes\n"
        "Assistant: Great! Please provide your contract ID so I can assist you with replacing an address from the existing list of addresses.\n"
        "\n"
        "Example 16:\n"
        "User: I selected the wrong number for the address.\n"
        "Assistant: No problem. Let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the correct address by entering the corresponding number.\n"
        "\n"
        "Example 17:\n"
        "User: 3\n"
        "Assistant: The number you selected is not valid. Please select a valid address number from the list:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Enter the number corresponding to the address you want to replace."
    ),
    ("user", "{user_input}"),
])


In [5]:
# Define the AddressDetails model
from langchain.memory import ConversationBufferMemory

class AddressDetails(BaseModel):
    """Extracts Entities such as Contract ID, full address, city, state, zipcode from the user input."""
    contract_id: str = Field(description="Contract ID")
    address_lane: str = Field(description="Address lane provided")
    city: str = Field(description="Name of the city")
    state: str = Field(description="Name of the state")
    zipcode: int = Field(description="Zipcode or the location")
# Initialize memory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history", return_context=True)
# Define the model
model = ChatOpenAI(temperature=0.0)
# Define the functions
extraction_functions = [convert_pydantic_to_openai_function(AddressDetails)]

In [6]:
# Bind the functions to the model
address_extraction_model = model.bind(functions=extraction_functions)
# Bind the functions to the model
address_extraction_model = model.bind(functions=extraction_functions)
# Create the chain
chain = replace_address_prompt | address_extraction_model | OpenAIFunctionsAgentOutputParser()

# Create the agent executor
agent_executor = AgentExecutor(agent=chain, tools=[], verbose=True, memory=memory)
# # Run the agent executor with input
# response = agent_executor.invoke({
#     "user_input": " "
# })

# print(response)

In [ ]:
# Run the agent executor with input
message = "I have slected to replace the address from an existing list, my contract ID is: "
print(message)
while True:
    user_input = input()
    if user_input in ['exit','bye']:
        break
    else:
        user_input = message +" "+ user_input
        response = agent_executor.invoke({
        "user_input": user_input
        })
        message = ""

print(response)

In [13]:
pip install langchain-community

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

ModuleNotFoundError: No module named 'langchain_community'

In [6]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('landlords.db')
cursor = conn.cursor()

# Create the table
cursor.execute('''
CREATE TABLE IF NOT EXISTS address (
    Contract_ID INTEGER,
    Date_of_Updation TEXT,
    Address TEXT
)
''')

# Insert dummy data
dummy_data = [
    (1, '2024-01-01', '123 Main St, Springfield'),
    (1, '2024-02-01', '124 Main St, Springfield'),
    (2, '2024-01-15', '456 Elm St, Shelbyville'),
    (2, '2024-03-20', '457 Elm St, Shelbyville'),
    (3, '2024-02-10', '789 Maple St, Capital City')
]

cursor.executemany('''
INSERT INTO address (Contract_ID, Date_of_Updation, Address)
VALUES (?, ?, ?)
''', dummy_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [7]:
# Function to query the table
def query_addresses(contract_id=None):
    # Connect to the SQLite database
    conn = sqlite3.connect('landlords.db')
    cursor = conn.cursor()

    if contract_id:
        # Query for a specific Contract_ID
        cursor.execute('SELECT * FROM address WHERE Contract_ID = ?', (contract_id,))
    else:
        # Query all records
        cursor.execute('SELECT * FROM address')
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Close the connection
    conn.close()
    
    return results

In [8]:

# Example usage
print("All records:")
for row in query_addresses():
    print(row)

All records:
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(2, '2024-01-15', '456 Elm St, Shelbyville')
(2, '2024-03-20', '457 Elm St, Shelbyville')
(3, '2024-02-10', '789 Maple St, Capital City')


In [ ]:


# Example usage
print("All records:")
for row in query_addresses():
    print(row)

print("\nRecords for Contract_ID 1:")
for row in query_addresses(1):
    print(row)

In [25]:
agent_executor.invoke({"user_input": ''})



> Entering new AgentExecutor chain...
Please provide me with the contract ID associated with the address you would like to replace.

> Finished chain.


{'user_input': 'did you fetch it ?',
 'chat_history': [HumanMessage(content='yes, i would like to '),
  AIMessage(content='Would you like to replace an existing address with a new one?'),
  HumanMessage(content='yes, i would like to '),
  AIMessage(content='Would you like to replace an existing address with a new one?'),
  HumanMessage(content='yes'),
  AIMessage(content='Would you like to replace an existing address with a new one?'),
  HumanMessage(content='replace'),
  AIMessage(content='Please provide me with the contract ID associated with the address you would like to replace.'),
  HumanMessage(content='abc1234'),
  AIMessage(content="I'm sorry, but I need a valid contract ID to proceed. Please provide the correct contract ID associated with the address you would like to replace."),
  HumanMessage(content='abc 1234'),
  AIMessage(content="I'm sorry, but I need a valid contract ID to proceed. Please provide the correct contract ID associated with the address you would like to repl

In [ ]:
# Define the combined chat prompt template for adding a new address
add_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with updating their address information. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Gather address information: address lane, city, state, and zip code.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of address changes.\n"
        "5. If the user asks a question unrelated to address changes, respond with: 'My functionality is limited to address changes.'\n"
        "\n"
        "If the user wants to add a new address, follow these steps:\n"
        "Step 1: Ask for the contract ID if it has not already been provided.\n"
        "Step 2: Ask for the full address including address lane, city, state (check if the state is located in the USA), and zip code.\n"
        "Step 3: Validate that all fields are provided. If any field is missing, prompt the user to provide the missing fields one by one.\n"
        "Step 4: Generate a reference ID and respond with: 'Your request for adding a new address is received. Your reference ID is [reference_id] for future communications. Thank you!'\n"
        "\n"
        "Example Conversation:\n"
        "Assistant: Can you provide me with your Contract ID?\n"
        "Human: ABC10023\n"
        "Assistant: Do you want to replace an existing address or add a new address?\n"
        "Human: I want to add an address.\n"
        "Assistant: You chose to add a new address. Please provide the full address including address lane, city, state, and zip code.\n"
        "User: 123 Main St, Springfield\n"
        "Assistant: It seems you missed the state and zip code. Please provide the state.\n"
        "Human: Illinois\n"
        "Assistant: Please provide the zip code.\n"
        "Human: 62704\n"
        "Assistant: Thank you. Your request for adding a new address is received. Your reference ID is [reference_id] for future communications. Thank you!\n"
    ),
    ("user", "{user_input}"),
])